In [ ]:
%cd ..

In [ ]:
import glob
import logging
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import quantities as pq

import epych

In [ ]:
%matplotlib inline

In [ ]:
logging.basicConfig(level=logging.INFO)

In [ ]:
CONDITIONS = ["go_gloexp", "go_seqctl", "lo_gloexp"]
PRETRIAL_SECONDS = 0.5
POSTTRIAL_SECONDS = 0.5

In [ ]:
def hippocampal_areas(probe_areas):
    for (c, a) in enumerate(probe_areas):
        if "DG-" in a or "CA" in a:
            yield (c, a)

def visual_areas(probe_areas):
    for (c, a) in enumerate(probe_areas):
        if "VIS" in a:
            yield (c, a)

def subcortical_areas(probe_areas):
    for (c, a) in enumerate(probe_areas):
        if "DG-" in a or "CA" in a or "MB" in a or "SCi" in a or "POST" in a:
            yield (c, a)

In [ ]:
NWB_FILES = glob.glob('/mnt/data/*.nwb')

In [ ]:
PILOT_FILES = []

In [ ]:
evoked = {}

In [ ]:
for cond in CONDITIONS:
    for f, nwb_file in enumerate(NWB_FILES):
        basename = os.path.splitext(nwb_file)[0]
        if not os.path.exists(basename + "_" + cond):
            continue
        sampling = epych.recording.Sampling.unpickle(basename + "_" + cond).smap(lambda sig: sig.median_filter())

        evoked[(cond, nwb_file, False)] = sampling.erp()
        evoked[(cond, nwb_file, True)] = sampling.baseline_correct(0., POSTTRIAL_SECONDS).erp()

In [ ]:
for cond in CONDITIONS:
    logging.info("Plotting baseline-corrected LFPs for %s across subjects" % cond)
    for f, nwb_file in enumerate(NWB_FILES):
        basename = os.path.splitext(nwb_file)[0]
        if not os.path.exists(basename + "_" + cond):
            continue
        evoked[(cond, nwb_file, True)].plot(vmin=-1e-4, vmax=1e-4, dpi=100, oddball_start=evoked[(cond, nwb_file, True)].trials["stim3_start"].mean())

In [ ]:
for cond in CONDITIONS:
    logging.info("Plotting baseline-corrected CSDs for %s across subjects" % cond)
    for f, nwb_file in enumerate(NWB_FILES):
        basename = os.path.splitext(nwb_file)[0]
        if not os.path.exists(basename + "_" + cond):
            continue
        evoked[(cond, nwb_file, True)].smap(lambda sig: sig.downsample(4).current_source_density(depth_column="vertical")).plot(vmin=-1e-4, vmax=1e-4, dpi=100, oddball_start=evoked[(cond, nwb_file, True)].trials["stim3_start"].mean())